In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# unscaled, log-normalized counts, with conditions subsampled to the same number of cells 
# and 2000 highly variable genes calculated jointly across all perturbation conditions, including control, using scanpy28 with default parameters (Supplementary Methods)

In [3]:
import scanpy as sc
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import os
from anndata import read_h5ad
import sys
sys.path.append("..")
from src import *

will use the CPU to calculate the distance matrix.
will use the CPU to calculate the distance matrix.


/data/bionets/je30bery/singularity/gt/lib/python3.13/site-packages/graph_tool/draw/cairo_draw.py:33: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/data/bionets/je30bery/singularity/gt/lib/python3.13/site-packages/graph_tool/draw/cairo_draw.py:33: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/data/bionets/je30bery/singularity/gt/lib/python3.13/site-packages/graph_tool/draw/cairo_draw.py:33: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/data/bionets/je30bery/singularity/gt/lib/python3.13/site-packages/graph_tool/all.py:39: RuntimeWarning: Error importing draw module, proceeding nevertheless: No module named 'cairo'
  warnings.warn(msg, RuntimeWarning)


In [4]:
data_path = "/mnt/data"

In [5]:
mcf = read_h5ad(os.path.join(data_path, 'mcfarland.hdf5'))

In [6]:
sc.pp.log1p(mcf)
sc.pp.highly_variable_genes(mcf, n_top_genes=2000)
mcf = mcf[:, mcf.var.highly_variable]

In [7]:
#sc.pp.neighbors(mcf, n_pcs=0)

In [8]:
g6 = list(np.unique([v for v in mcf.obs["perturbation"].values if v.endswith("_6")]))
g24 = list(np.unique([v for v in mcf.obs["perturbation"].values if v.endswith("_24")]))
test_groups = [g6, g24]
reference = "control"

In [9]:
rosenbaum(mcf, group_by="perturbation", reference=reference, test_group=g6, use_nx=False, k=None)

/mnt/je30bery/rosenbaum_test/notebooks/../src/rosenbaum.py:153: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["XMatch_group"] = np.where(adata.obs[group_by].isin(test_group), "test", "reference")


XMatch_group
reference    541
test         392
Name: count, dtype: int64
using CPU to calculate distance matrix.
creating distance graph with 934 samples
num samples 933 edges 436645
matching.
NUM 933
counting cross matches.


(8.141915891577072e-38, np.float64(-12.56855421634986), 1.0)

In [10]:
rosenbaum(mcf, group_by="perturbation", reference=reference, test_group=g6, use_nx=False, k=5)

will use the CPU to calculate the distance matrix.
XMatch_group
reference    541
test         392
Name: count, dtype: int64
calculating PCA and kNN graph.


/mnt/je30bery/rosenbaum_test/notebooks/../src/rosenbaum.py:153: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["XMatch_group"] = np.where(adata.obs[group_by].isin(test_group), "test", "reference")
/data/bionets/je30bery/singularity/gt/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


counting cross matches.


(1.87702863286868e-38, np.float64(-12.669890894843123), 1.0)

In [11]:
rosenbaum(mcf, group_by="perturbation", reference=reference, test_group=g6, use_nx=False, k=2)

/mnt/je30bery/rosenbaum_test/notebooks/../src/rosenbaum.py:153: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["XMatch_group"] = np.where(adata.obs[group_by].isin(test_group), "test", "reference")


XMatch_group
reference    541
test         392
Name: count, dtype: int64
calculating PCA and kNN graph.
counting cross matches.


(8.64187385289462e-33, np.float64(-11.624933449879777), 0.759656652360515)

In [12]:
rosenbaum(mcf, group_by="perturbation", reference=reference, test_group=g6, use_nx=False, k=2)

/mnt/je30bery/rosenbaum_test/notebooks/../src/rosenbaum.py:153: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["XMatch_group"] = np.where(adata.obs[group_by].isin(test_group), "test", "reference")


XMatch_group
reference    541
test         392
Name: count, dtype: int64
calculating PCA and kNN graph.
counting cross matches.


(8.64187385289462e-33, np.float64(-11.624933449879777), 0.759656652360515)